In [1]:
import numpy as np
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

/Users/jennylee/opt/anaconda3/lib/python3.9/site-packages/pdpipe/col_generation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
%store -r Before_labelling
df = Before_labelling.copy()
df.head()

,ProdId,Description,SalesGroup,Weight (g),GHG Emission (g),N lost (g),Freshwater Withdrawals (L),Stress-Weighted Water Use (L),GHG Emission (g) / 100g,N lost (g) / 100g,Freshwater Withdrawals (L) / 100g,Stress-Weighted Water Use (L) / 100g
0,R-61778,ALF|Flatbread|4 Cheese,OK - AL FORNO,185.0,1229.947500,11.939000,212.84,10662.59,664.84,6.45,115.05,5763.56
1,R-61780,ALF|Flatbread|Apple & Pancetta,OK - AL FORNO,140.0,756.345750,9.031250,144.99,5287.95,540.25,6.45,103.56,3777.11
2,R-61749,ALF|Flatbread|BBQ Chicken,OK - AL FORNO,245.0,1011.756923,18.958191,118.17,3196.50,412.96,7.74,48.23,1304.69
3,R-50859,ALF|Flatbread|Bruschetta,OK - AL FORNO,215.0,454.128829,4.589045,75.98,3520.38,211.22,2.13,35.34,1637.39
4,R-50788,ALF|Flatbread|Caprese,OK - AL FORNO,233.0,1010.276452,10.809234,150.95,7671.61,433.60,4.64,64.79,3292.54


In [3]:
df.shape

(458, 12)

In [4]:
fifty_cutoff = 180.12
overall_baseline = 360.25

In [5]:
def create_ghg_label(label):
    if label > overall_baseline:
        return "Red"
    elif (label <= overall_baseline) & (label > fifty_cutoff):
        return "Yellow"
    elif label <= fifty_cutoff:
        return "Green"

df["Label"] = df["GHG Emission (g) / 100g"].apply(lambda x: create_ghg_label(x))
df.head()

,ProdId,Description,SalesGroup,Weight (g),GHG Emission (g),N lost (g),Freshwater Withdrawals (L),Stress-Weighted Water Use (L),GHG Emission (g) / 100g,N lost (g) / 100g,Freshwater Withdrawals (L) / 100g,Stress-Weighted Water Use (L) / 100g,Label
0,R-61778,ALF|Flatbread|4 Cheese,OK - AL FORNO,185.0,1229.947500,11.939000,212.84,10662.59,664.84,6.45,115.05,5763.56,Red
1,R-61780,ALF|Flatbread|Apple & Pancetta,OK - AL FORNO,140.0,756.345750,9.031250,144.99,5287.95,540.25,6.45,103.56,3777.11,Red
2,R-61749,ALF|Flatbread|BBQ Chicken,OK - AL FORNO,245.0,1011.756923,18.958191,118.17,3196.50,412.96,7.74,48.23,1304.69,Red
3,R-50859,ALF|Flatbread|Bruschetta,OK - AL FORNO,215.0,454.128829,4.589045,75.98,3520.38,211.22,2.13,35.34,1637.39,Yellow
4,R-50788,ALF|Flatbread|Caprese,OK - AL FORNO,233.0,1010.276452,10.809234,150.95,7671.61,433.60,4.64,64.79,3292.54,Red


In [6]:
def create_results_GHG(df):
    num_red = df["Label"].value_counts()["Red"]
    num_yellow = df["Label"].value_counts()["Yellow"]
    num_green = df["Label"].value_counts()["Green"]
    results = pd.DataFrame([num_green, num_yellow, num_red], index=["Green", "Yellow", "Red"], columns=["Count"])
    return results

create_results_GHG(df)

,Count
Green,159
Yellow,163
Red,136


In [7]:
GHG_baseline = 381.13
nitrogen_baseline = 4.21
water_baseline = 1501.2

df["Combined Label"] = ""

def create_results_all_factors(df):
    
    def calculate_all_factors(ghg, nitrogen, water):
        weighted_ghg = ghg / (3 * GHG_baseline)
        weighted_nitrogen = nitrogen / (3 * nitrogen_baseline)
        weighted_water = water / (3 * water_baseline)
        return weighted_ghg + weighted_nitrogen + weighted_water

    for ind, row in df.iterrows():
        label = calculate_all_factors(row["GHG Emission (g) / 100g"], row["N lost (g) / 100g"], row["Stress-Weighted Water Use (L) / 100g"])
        if label <= 0.5:
            df.loc[ind, "Combined Label"] = True
        else:
            df.loc[ind, "Combined Label"] = False
            
    return df

In [8]:
final_result = create_results_all_factors(df)
final_result 

,ProdId,Description,SalesGroup,Weight (g),GHG Emission (g),N lost (g),Freshwater Withdrawals (L),Stress-Weighted Water Use (L),GHG Emission (g) / 100g,N lost (g) / 100g,Freshwater Withdrawals (L) / 100g,Stress-Weighted Water Use (L) / 100g,Label,Combined Label
0,R-61778,ALF|Flatbread|4 Cheese,OK - AL FORNO,185.000000,1229.947500,11.939000,212.84,10662.59,664.84,6.45,115.05,5763.56,Red,False
1,R-61780,ALF|Flatbread|Apple & Pancetta,OK - AL FORNO,140.000000,756.345750,9.031250,144.99,5287.95,540.25,6.45,103.56,3777.11,Red,False
2,R-61749,ALF|Flatbread|BBQ Chicken,OK - AL FORNO,245.000000,1011.756923,18.958191,118.17,3196.50,412.96,7.74,48.23,1304.69,Red,False
3,R-50859,ALF|Flatbread|Bruschetta,OK - AL FORNO,215.000000,454.128829,4.589045,75.98,3520.38,211.22,2.13,35.34,1637.39,Yellow,False
4,R-50788,ALF|Flatbread|Caprese,OK - AL FORNO,233.000000,1010.276452,10.809234,150.95,7671.61,433.60,4.64,64.79,3292.54,Red,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,R-57815,SQR|Tofu Sofrito Quesadilla +1,OK - SQUARE MEAL,650.000002,1542.748135,11.964991,171.88,8616.10,237.35,1.84,26.44,1325.55,Yellow,False
454,R-61679,SQR|Tofu Sofrito Quesadilla +2,OK - SQUARE MEAL,1065.000002,2275.218575,16.560511,214.99,11264.83,213.64,1.55,20.19,1057.73,Yellow,False
455,R-56902,SQR|Vegan Lettuce Wrap,OK - SQUARE MEAL,399.999993,640.295633,3.987816,81.04,4682.22,160.07,1.00,20.26,1170.56,Green,True
456,R-57810,SQR|Vegan Lettuce Wrap +1,OK - SQUARE MEAL,544.999993,806.712203,4.920096,157.61,5354.54,148.02,0.90,28.92,982.48,Green,True


In [9]:
final_result["All Label"] = ""

def create_all_label(df):
    
    for ind, row in df.iterrows(): 
        if (row["Label"] == "Green") & (row["Combined Label"] == True):
            df.loc[ind, "All Label"] = "Green"
        elif (row["Label"] == "Green") & (row["Combined Label"] == False):
            df.loc[ind, "All Label"] = "Yellow"
        elif (row["Label"] != "Green") & (row["Combined Label"] == False):
            df.loc[ind, "All Label"] = "Red"
        
    return df

final_results2 = create_all_label(final_result)
final_results2

,ProdId,Description,SalesGroup,Weight (g),GHG Emission (g),N lost (g),Freshwater Withdrawals (L),Stress-Weighted Water Use (L),GHG Emission (g) / 100g,N lost (g) / 100g,Freshwater Withdrawals (L) / 100g,Stress-Weighted Water Use (L) / 100g,Label,Combined Label,All Label
0,R-61778,ALF|Flatbread|4 Cheese,OK - AL FORNO,185.000000,1229.947500,11.939000,212.84,10662.59,664.84,6.45,115.05,5763.56,Red,False,Red
1,R-61780,ALF|Flatbread|Apple & Pancetta,OK - AL FORNO,140.000000,756.345750,9.031250,144.99,5287.95,540.25,6.45,103.56,3777.11,Red,False,Red
2,R-61749,ALF|Flatbread|BBQ Chicken,OK - AL FORNO,245.000000,1011.756923,18.958191,118.17,3196.50,412.96,7.74,48.23,1304.69,Red,False,Red
3,R-50859,ALF|Flatbread|Bruschetta,OK - AL FORNO,215.000000,454.128829,4.589045,75.98,3520.38,211.22,2.13,35.34,1637.39,Yellow,False,Red
4,R-50788,ALF|Flatbread|Caprese,OK - AL FORNO,233.000000,1010.276452,10.809234,150.95,7671.61,433.60,4.64,64.79,3292.54,Red,False,Red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,R-57815,SQR|Tofu Sofrito Quesadilla +1,OK - SQUARE MEAL,650.000002,1542.748135,11.964991,171.88,8616.10,237.35,1.84,26.44,1325.55,Yellow,False,Red
454,R-61679,SQR|Tofu Sofrito Quesadilla +2,OK - SQUARE MEAL,1065.000002,2275.218575,16.560511,214.99,11264.83,213.64,1.55,20.19,1057.73,Yellow,False,Red
455,R-56902,SQR|Vegan Lettuce Wrap,OK - SQUARE MEAL,399.999993,640.295633,3.987816,81.04,4682.22,160.07,1.00,20.26,1170.56,Green,True,Green
456,R-57810,SQR|Vegan Lettuce Wrap +1,OK - SQUARE MEAL,544.999993,806.712203,4.920096,157.61,5354.54,148.02,0.90,28.92,982.48,Green,True,Green


In [10]:
for ind, row in final_results2.iterrows():
    if "Veg Burger" in row["Description"]:
        print(row["Description"])

GRL|Veg Burger|Black Bean
GRL|Veg Burger|Chickpea
GRL|Veg Burger|Lentil
GRL|Veg Burger|Portobello
GRL|Veg Burger|Ultimate


In [11]:
OK = pd.read_excel("menus/Open-Kitchen/Open Kitchen Menu Grid Aug.-Dec. 2022.xlsx", sheet_name=1)
OK.shape

FileNotFoundError: [Errno 2] No such file or directory: 'menus/Open-Kitchen/Open Kitchen Menu Grid Aug.-Dec. 2022.xlsx'

In [ ]:
OK_adjusted = OK.iloc[:, 1:18]
OK_adjusted = OK_adjusted.dropna()
OK_adjusted

In [ ]:
list(OK_adjusted)

In [ ]:
menu_list = []
all_ind = OK_adjusted.index.values.tolist()

for item in list(OK_adjusted):
    temp_list = OK_adjusted[item].tolist()
    for item_2 in temp_list:
        if item_2 not in menu_list:
            menu_list.append(item_2)

menu_list

In [ ]:
for ind, row in final_results2.iterrows():
    for item in menu_list:
        if item in row["Description"]:
            print(row["Description"])

In [ ]:
OK2 = pd.ExcelFile("menus/Open-Kitchen/Open Kitchen Menu Grid Aug.-Dec. 2022.xlsx")
lens = len(OK2.sheet_names)
lens

In [ ]:
all_menus = []

def load_all_menus(excel_file):
    file = pd.ExcelFile(excel_file)
    sheet_number = len(file.sheet_names)
    for sheet in range(sheet_number):
        read_sheet = pd.read_excel(excel_file, sheet_name=sheet)
        sheet_shape = read_sheet.shape
        print(sheet_shape[1])
#         read_sheet = sheet.iloc[:, 1:sheet_shape[1]-1]
#         print(read_sheet)
    
load_all_menus("menus/Open-Kitchen/Open Kitchen Menu Grid Aug.-Dec. 2022.xlsx")